In [1]:
pip install nltk

     |████████████████████████████████| 1.4MB 7.3MB/s eta 0:00:01
     |████████████████████████████████| 307kB 7.4MB/s eta 0:00:01
     |████████████████████████████████| 675kB 7.0MB/s eta 0:00:01
     |████████████████████████████████| 81kB 4.4MB/s  eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk import word_tokenize, pos_tag, pos_tag_sents

### Import modules needed

In [3]:
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger') 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords


from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet 
# Create WordNetLemmatizer object 

sid = SentimentIntensityAnalyzer()
lemmatizer = WordNetLemmatizer() 

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### Customized POS Tagger

In [4]:
def pos_tagger(nltk_tag): 
    if nltk_tag.startswith('J'): 
        return wordnet.ADJ 
    elif nltk_tag.startswith('V'): 
        return wordnet.VERB 
    elif nltk_tag.startswith('N'): 
        return wordnet.NOUN 
    elif nltk_tag.startswith('R'): 
        return wordnet.ADV 
    else:           
        return None

In [5]:
df = pd.read_csv('opinion_survey.csv')
df = df.replace(np.nan, 'Neutral', regex=True) ### Some values are missing, use "Neutral" for missing values
df = df.drop(df.columns[[0]],axis=1) ### Drop extra index
test_df = df.iloc[0:, 0:] # Just for renaming purposes so that df variable can be used for testing
test_df

,I can easily log-in and log-out my Canvas account. (Please write your views relating your answer above.),I can access my course using the mobile Canvas app. (Please write your views about this relating to your answer above.),I can access my Canvas account using the available computers in JRU. (Please write your views about this relating to your answer above.),I have no issues accessing my Canvas account using the wired internet connection in the computer laboratories. (Please write your views about this relating to your answer above.),I can easily get help when I have problems with my Canvas account. (Please write your views about this relating to your answer above.),I can access my Canvas account outside JRU. (Please write your views about this relating to your answer above.),I can clearly understand the lessons on my Canvas modules. (Please write your views about this relating to your answer above.),I recognize that the modules in my course are well organized. (Please write your views about this relating to your answer above.),I can easily manage the navigation of the courses through the desktop version. .1,I can easily manage the navigation of the courses through the mobile app. (Please write your views about this relating to your answer above.),...,I understand the lessons better with the help of course materials in Canvas. (Please write your views about this relating to your answer above.),Canvas course materials make me interested. (Please write your views about this relating to your answer above.),"The course materials on Canvas helped me prepare for: Face to face discussions, quizzes, major exams, assignments, forums, etc. (Please write your views about this relating to your answer above.)",My instructor knows how to use Canvas. (Please write your views about this relating to your answer above.),My instructor demonstrated mastery of the subject matter. (Please write your views about this relating to your answer above.),My instructor encourages me to use Canvas. (Please write your views about this relating to your answer above.),"Canvas has made it easy for me to: communicate with my instructor, receive class announcements, share ideas on group discussions","I can answer the assessment materials (quiz, exam) posted in Canvas. (Please write your views about this relating to your answer above.)","I find the course materials on Canvas aligned with the: quizzes, major exams, assignments, forums, etc.) (Please write your views about this relating to your answer above.)",I have been give enough time to accomplish the activities on Canvas. (Please write your views about this relating to your answer above.)
0,i can,i can,i can,i can,i can,i can,i can,i can,i can,i can,...,i can,i can,i can,i can,i can,i can,i can,i can,i can,-
1,i have no problems in log in,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,...,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem
2,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,...,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,"Yes, I have no problems with it"
3,i dont have problem in my canvas,i can easily login my canvas app in my mobile app,I can access my Canvas account using the avail...,i dont have problem in internet connection,i dont have problem with it,i dont have problem with it,i dont have problem with it,i dont have problem with it,i d

In [6]:
stoplist = stopwords.words('english') + ['though']
def remove_stopWords(w): 
    w = ' '.join(word for word in w.split() if word not in stoplist)
    return w

In [7]:
col_range = len(test_df.columns) # number of columns

for i in range(0,col_range):
    col = test_df.columns[i] # The current column
    test_df.loc[:,col] = test_df[col].apply(lambda x : str.lower(str(x))) ## To Lower Case
    test_df.loc[:,col] = test_df[col].apply(lambda x : " ".join(re.findall('[\w]+',x))) # Remove Punctuations
    test_df.loc[:,col] = test_df[col].apply(lambda x : remove_stopWords(x)) # Remove Stop words
    
    ##POS TAGGING
    texts = test_df.loc[:,col].tolist()
    tagged_texts = pos_tag_sents(map(word_tokenize, texts)) ### Tag every word in a row with POS
    
    ### Lemmatization
    new = []
    for i in tagged_texts:
        #if len(i) > 0:
        lemmatized_sentence = []
        for word, tag in i:
            tag = pos_tagger(tag) ### Convert POS Tag to known POS for simplification
            if tag is None: 
    # if there is no available tag, append the token as is 
                lemmatized_sentence.append(word) 
            else:         
    # else use the tag to lemmatize the token 
                lemmatized_sentence.append(lemmatizer.lemmatize(word, tag)) 

        lemmatized_sentence = " ".join(lemmatized_sentence) 
        #print(lemmatized_sentence)
        new.append(lemmatized_sentence)
    else:
        pass
    

    test_df['POS'] = new ## Store tagged words
    
test_df = test_df.replace(r'^\s*$', "neutral", regex=True) ## If row value is null, replace with neutral string
test_df

,I can easily log-in and log-out my Canvas account. (Please write your views relating your answer above.),I can access my course using the mobile Canvas app. (Please write your views about this relating to your answer above.),I can access my Canvas account using the available computers in JRU. (Please write your views about this relating to your answer above.),I have no issues accessing my Canvas account using the wired internet connection in the computer laboratories. (Please write your views about this relating to your answer above.),I can easily get help when I have problems with my Canvas account. (Please write your views about this relating to your answer above.),I can access my Canvas account outside JRU. (Please write your views about this relating to your answer above.),I can clearly understand the lessons on my Canvas modules. (Please write your views about this relating to your answer above.),I recognize that the modules in my course are well organized. (Please write your views about this relating to your answer above.),I can easily manage the navigation of the courses through the desktop version. .1,I can easily manage the navigation of the courses through the mobile app. (Please write your views about this relating to your answer above.),...,Canvas course materials make me interested. (Please write your views about this relating to your answer above.),"The course materials on Canvas helped me prepare for: Face to face discussions, quizzes, major exams, assignments, forums, etc. (Please write your views about this relating to your answer above.)",My instructor knows how to use Canvas. (Please write your views about this relating to your answer above.),My instructor demonstrated mastery of the subject matter. (Please write your views about this relating to your answer above.),My instructor encourages me to use Canvas. (Please write your views about this relating to your answer above.),"Canvas has made it easy for me to: communicate with my instructor, receive class announcements, share ideas on group discussions","I can answer the assessment materials (quiz, exam) posted in Canvas. (Please write your views about this relating to your answer above.)","I find the course materials on Canvas aligned with the: quizzes, major exams, assignments, forums, etc.) (Please write your views about this relating to your answer above.)",I have been give enough time to accomplish the activities on Canvas. (Please write your views about this relating to your answer above.),POS
0,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,...,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral
1,problems log,problem,problem,problem,problem,problem,problem,problem,problem,problem,...,problem,problem,problem,problem,problem,problem,problem,problem,problem,problem
2,problems,problems,problems,problems,problems,problems,problems,problems,problems,problems,...,problems,problems,problems,problems,problems,problems,problems,problems,yes problems,yes problem
3,dont problem canvas,easily login canvas app mobile app,access canvas account using available computer...,dont problem internet connection,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,...,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem
4,problems,access course,problem accessing canvas,dont issue,problem,problem,understand,easy recognize,manage,manage,...,im interested,course materials helped,yes teach us,yes demonstrate,encourage,communicate easy,problem,problem,problem,problem
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,problem,problem,yes access,sometimes takes time load,yes people office always ready help,problem,yes clearly,yes well organized,problem,problem,...,interesting,problem,yes knows,yes teaching well,yes forums need answer,problem,yes,problem,yes sometimes extends dead

In [8]:
test_df.to_csv('lemmatized_opinion.csv', index=False)

### Sample POS Tags from first row

In [9]:
from pprint import pprint 

pprint(tagged_texts[:20])

[[],
 [('problem', 'NN')],
 [('yes', 'RB'), ('problems', 'NNS')],
 [('dont', 'NN'), ('problem', 'NN')],
 [('problem', 'NN')],
 [('yes', 'NNS'), ('course', 'NN')],
 [('give', 'VB'), ('week', 'NN'), ('deadline', 'NN'), ('activity', 'NN')],
 [('deadline', 'NN'),
  ('every', 'DT'),
  ('activities', 'NNS'),
  ('able', 'JJ'),
  ('us', 'PRP'),
  ('late', 'JJ')],
 [('sometimes', 'RB'),
  ('know', 'VBP'),
  ('due', 'JJ'),
  ('date', 'NN'),
  ('finish', 'JJ'),
  ('time', 'NN')],
 [('yes', 'RB'), ('given', 'VBN'), ('enough', 'JJ'), ('time', 'NN')],
 [('yes', 'RB'), ('especially', 'RB'), ('weekends', 'NNS')],
 [('give', 'VB'),
  ('enough', 'JJ'),
  ('time', 'NN'),
  ('accomplish', 'JJ'),
  ('activities', 'NNS'),
  ('canvas', 'VBP')],
 [('takes', 'VBZ'),
  ('time', 'NN'),
  ('week', 'NN'),
  ('submit', 'NN'),
  ('activities', 'NNS')],
 [('yes', 'NNS')],
 [('give', 'VB'),
  ('enough', 'JJ'),
  ('time', 'NN'),
  ('accomplish', 'JJ'),
  ('activities', 'NNS'),
  ('canvas', 'VBP')],
 [('give', 'VB'),
  

In [10]:
lem_df = pd.read_csv('lemmatized_opinion.csv')
lem_df = lem_df.iloc[0:, 0:-1] # Remove last columns
lem_df

,I can easily log-in and log-out my Canvas account. (Please write your views relating your answer above.),I can access my course using the mobile Canvas app. (Please write your views about this relating to your answer above.),I can access my Canvas account using the available computers in JRU. (Please write your views about this relating to your answer above.),I have no issues accessing my Canvas account using the wired internet connection in the computer laboratories. (Please write your views about this relating to your answer above.),I can easily get help when I have problems with my Canvas account. (Please write your views about this relating to your answer above.),I can access my Canvas account outside JRU. (Please write your views about this relating to your answer above.),I can clearly understand the lessons on my Canvas modules. (Please write your views about this relating to your answer above.),I recognize that the modules in my course are well organized. (Please write your views about this relating to your answer above.),I can easily manage the navigation of the courses through the desktop version. .1,I can easily manage the navigation of the courses through the mobile app. (Please write your views about this relating to your answer above.),...,I understand the lessons better with the help of course materials in Canvas. (Please write your views about this relating to your answer above.),Canvas course materials make me interested. (Please write your views about this relating to your answer above.),"The course materials on Canvas helped me prepare for: Face to face discussions, quizzes, major exams, assignments, forums, etc. (Please write your views about this relating to your answer above.)",My instructor knows how to use Canvas. (Please write your views about this relating to your answer above.),My instructor demonstrated mastery of the subject matter. (Please write your views about this relating to your answer above.),My instructor encourages me to use Canvas. (Please write your views about this relating to your answer above.),"Canvas has made it easy for me to: communicate with my instructor, receive class announcements, share ideas on group discussions","I can answer the assessment materials (quiz, exam) posted in Canvas. (Please write your views about this relating to your answer above.)","I find the course materials on Canvas aligned with the: quizzes, major exams, assignments, forums, etc.) (Please write your views about this relating to your answer above.)",I have been give enough time to accomplish the activities on Canvas. (Please write your views about this relating to your answer above.)
0,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,...,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral
1,problems log,problem,problem,problem,problem,problem,problem,problem,problem,problem,...,problem,problem,problem,problem,problem,problem,problem,problem,problem,problem
2,problems,problems,problems,problems,problems,problems,problems,problems,problems,problems,...,problems,problems,problems,problems,problems,problems,problems,problems,problems,yes problems
3,dont problem canvas,easily login canvas app mobile app,access canvas account using available computer...,dont problem internet connection,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,...,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem,dont problem
4,problems,access course,problem accessing canvas,dont issue,problem,problem,understand,easy recognize,manage,manage,...,understand lesson,im interested,course materials helped,yes teach us,yes demonstrate,encourage,communicate easy,problem,problem,problem
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,problem,problem,yes access,sometimes takes time load,yes people office always ready help,problem,yes clearly,yes well

In [17]:
comp = []
col_range = len(lem_df.columns) # number of columns

for i in range(0,col_range):
    col = test_df.columns[i] # The current column
    test_df['scores'] = test_df[col].apply(lambda x: sid.polarity_scores(x)) ## Get polarity score of every Column
    #compound = test_df['scores'].apply(lambda score_dict: score_dict['compound']) ## Extract the compound from the results
    #test_df = test_df.drop('scores', 1) # Drop score DF in every iteration
    #compound = sum(compound)/140 # Get the mean compound of each columns
    #comp.append(compound) # Save mean and append to list
    

## Show Neg/Neu/Pos

In [30]:
pd.set_option('display.max_rows',test_df.shape[0]+1)
pd.set_option('display.max_columns', None)
print(test_df['scores'].to_frame())

                                                scores
0    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
1    {'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound...
2    {'neg': 0.5, 'neu': 0.0, 'pos': 0.5, 'compound...
3    {'neg': 0.0, 'neu': 0.307, 'pos': 0.693, 'comp...
4    {'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound...
5    {'neg': 0.0, 'neu': 0.27, 'pos': 0.73, 'compou...
6    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
7    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
8    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
9    {'neg': 0.0, 'neu': 0.526, 'pos': 0.474, 'comp...
10   {'neg': 0.0, 'neu': 0.426, 'pos': 0.574, 'comp...
11   {'neg': 0.0, 'neu': 0.641, 'pos': 0.359, 'comp...
12   {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
13   {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...
14   {'neg': 0.0, 'neu': 0.641, 'pos': 0.359, 'comp...
15   {'neg': 0.0, 'neu': 0.641, 'pos': 0.359, 'comp...
16   {'neg': 0.0, 'neu': 0.641, 'pos': 0.359, 'comp...
17   {'neg

In [22]:
from pprint import pprint



[0      {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
1      {'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound...
2      {'neg': 0.5, 'neu': 0.0, 'pos': 0.5, 'compound...
3      {'neg': 0.0, 'neu': 0.307, 'pos': 0.693, 'comp...
4      {'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound...
                             ...                        
135    {'neg': 0.0, 'neu': 0.323, 'pos': 0.677, 'comp...
136    {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...
137    {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...
138    {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...
139    {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...
Name: scores, Length: 140, dtype: object]


## Comparisons

In [13]:
df_num = pd.read_csv('opinion_sentiment_score.csv')
cor_num = df_num.iloc[0:, 0:7:6]
cor_num


,index,Sentiment Score
0,I can easily log-in and log-out my Canvas acco...,0.148826
1,I can access my course using the mobile Canvas...,0.140744
2,I can access my Canvas account using the avail...,0.141499
3,I can easily get help when I have problems wit...,-0.007863
4,I have no issues accessing my Canvas account u...,0.244604
5,I can access my Canvas account outside JRU.,0.155105
6,I can clearly understand the lessons on my Can...,0.228990
7,I recognize that the modules in my course are ...,0.242035
8,I can easily manage the navigation of the cour...,0.283602
9,I can easily manage the navigation of the cour...,0.210798


In [14]:
df_ngram = pd.read_csv('ngram_sentiment_score.csv')
#ngram_list = df_ngram.values.tolist()
ngram_list = df_ngram.values.flatten()

In [15]:

cor_num = cor_num.rename({'Sentiment Score': 'Without Lemmanization'}, axis=1) 
cor_num["With Lemmanization"] = comp
cor_num["Lemmatized and Ngram"] = ngram_list

cor_num.style.set_caption('Comparison of Sentiment Score results')

,index,Without Lemmanization,With Lemmanization,Lemmatized and Ngram
0,I can easily log-in and log-out my Canvas account.,0.148826,0.171376,0.097426
1,I can access my course using the mobile Canvas app.,0.140744,0.163131,0.086712
2,I can access my Canvas account using the available computers in JRU.,0.141499,0.159206,0.084185
3,I can easily get help when I have problems with my Canvas account.,-0.007863,0.084881,0.026009
4,I have no issues accessing my Canvas account using the wired internet connection in the computer laboratories.,0.244604,0.271197,0.098726
5,I can access my Canvas account outside JRU.,0.155105,0.176365,0.068340
6,I can clearly understand the lessons on my Canvas modules.,0.228990,0.266153,0.155156
7,I recognize that the modules in my course are well organized.,0.242035,0.273417,0.203494
8,I can easily manage the navigation of the courses through the desktop version.,0.283602,0.316241,0.246295
9,I can easily manage the navigation of the courses through the mobile app.,0.210798,0.251833,0.118517


In [16]:
cor_num.to_csv("Sentiment_Comparison.csv",index=False)